In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
#import keras
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Dense, Dropout
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
ia.seed(1)

sometimes = lambda aug: iaa.Sometimes(0.3, aug)

# Define our sequence of augmentation steps that will be applied to every image.
aug_seq = iaa.Sequential([ iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop some of the images by 0-10% of their height/width
        sometimes(iaa.Crop(percent=(0, 0.1))),
        # Apply affine transformations to some of the images
        sometimes(iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-45, 45), shear=(-16, 16), order=[0, 1], cval=(0, 255), mode=ia.ALL)), iaa.SomeOf((0, 5),
        # Convert some images into their superpixel representation
        [sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))),
        # Blur each image with varying strength using
        iaa.OneOf([iaa.GaussianBlur((0, 3.0)),iaa.AverageBlur(k=(2, 7)), iaa.MedianBlur(k=(3, 11))]),
        # Sharpen each image, overlay the result with the original
         iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
         # Same as sharpen, but for an embossing effect.
         iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
         # Search in some images either for all edges or for directed edges.
         sometimes(iaa.OneOf([iaa.EdgeDetect(alpha=(0, 0.7)),iaa.DirectedEdgeDetect(alpha=(0, 0.7), direction=(0.0, 1.0))])),
         # Add gaussian noise to some images.
         iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
         # Either drop randomly 1 to 10% of all pixels (i.e. set them to black) or drop them on an image with 2-5% percent
         # of the original size, leading to large dropped rectangles.
         iaa.OneOf([iaa.Dropout((0.01, 0.1), per_channel=0.5),iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05),
        per_channel=0.2)]),
         # Invert each image's chanell with 5% probability.
         iaa.Invert(0.05, per_channel=True), # invert color channels
         # Add a value of -10 to 10 to each pixel.
         iaa.Add((-10, 10), per_channel=0.5),
         # Change brightness of images (50-150% of original value).
         iaa.Multiply((0.5, 1.5), per_channel=0.5),
         # Improve or worsen the contrast of images.
         iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5),
         # Convert each image to grayscale and then overlay the
         iaa.Grayscale(alpha=(0.0, 1.0)),
         # In some images move pixels locally around (with random strengths).
         sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)),
         # In some images distort local areas with varying strength.
         sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05)))],random_order=True)], random_order=True)

In [0]:
augidx = np.random.randint(0,50000, size=(20000)) #an array of 20k random indexes between (0,50k)

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

x_aug = aug_seq.augment_images(x_train[augidx]) #generating 20k augmented images with augindexes
y_aug= y_train[augidx]

x_train = np.append(x_train, x_aug, axis=0) #new training set consisting of augment images & labels
y_train = np.append(y_train, y_aug, axis=0)

In [0]:
#convert integers to float; normalise and center the mean
x_train=x_train.astype("float32")  
x_test=x_test.astype("float32")
mean=np.mean(x_train)
std=np.std(x_train)
x_test=(x_test-mean)/std
x_train=(x_train-mean)/std

num_classes = 10

In [0]:
# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [36]:
print('shapes of x_train & y_train \n',x_train.shape, y_train.shape)

shapes of x_train & y_train 
 (70000, 32, 32, 3) (70000, 10)


In [0]:
# Hyperparameters
batch_size = 128
#epochs = 125
num_filter = 64
compression = 0.5
dropout_rate = 0.2

In [0]:
# Dense Block
def add_denseblock(input, num_filter, dropout_rate, layers, growth_rate, grow_nb_filters = False):
  
  global compression
  temp = input
  for _ in range(layers):
    
    BatchNorm = BatchNormalization()(temp)
    relu = Activation('relu')(BatchNorm)

    Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3),
                        use_bias=False ,padding='same')(relu)
    if dropout_rate:
      Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
    concat = Concatenate(axis=-1)([temp,Conv2D_3_3])

    temp = concat

    if grow_nb_filters:
      num_filter += growth_rate  

  return temp, num_filter

In [0]:
def add_transition(input, num_filter, dropout_rate):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression),
                               (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
dropout_rate = 0.2
l = 12
growth_rate = 16
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block, num_filter = add_denseblock(First_Conv2D, num_filter, dropout_rate + 0.1 , 8, growth_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)
num_filter = int(num_filter * compression)


Second_Block, num_filter = add_denseblock(First_Transition, num_filter, dropout_rate, 6, growth_rate, True)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)
num_filter = int(num_filter * compression)


Third_Block, num_filter = add_denseblock(Second_Transition, num_filter, dropout_rate, 4, growth_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)
num_filter = int(num_filter * compression)

Last_Block, num_filter = add_denseblock(Third_Transition,  num_filter, dropout_rate, 4, growth_rate)
output = output_layer(Last_Block)


In [42]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
datagen = ImageDataGenerator(rotation_range=15,
                             horizontal_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.3
                             )

In [0]:
datagen.fit(x_train)

In [0]:
# determine Loss function and Optimizer
opt_adam = keras.optimizers.Adam(lr=0.001,decay=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['accuracy'])

In [21]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),
                    steps_per_epoch = x_train.shape[0] // batch_size,
                    epochs=80, verbose =1, validation_data =(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Epoch 1/80
546/546 [==============================] - 290s 531ms/step - loss: 1.6436 - acc: 0.4063 - val_loss: 1.3333 - val_acc: 0.5335
Epoch 2/80
546/546 [==============================] - 281s 514ms/step - loss: 1.3424 - acc: 0.5232 - val_loss: 1.5101 - val_acc: 0.5377
Epoch 3/80
546/546 [==============================] - 279s 511ms/step - loss: 1.1831 - acc: 0.5849 - val_loss: 0.9923 - val_acc: 0.6463
Epoch 4/80
546/546 [==============================] - 279s 511ms/step - loss: 1.0855 - acc: 0.6184 - val_loss: 1.1539 - val_acc: 0.6400
Epoch 5/80
546/546 [==============================] - 279s 510ms/step - loss: 1.0116 - acc: 0.6462 - val_loss: 0.7522 - val_acc: 0.7437
Epoch 6/80
546/546 [==============================] - 279s 511ms/step - loss: 0.9599 - acc: 0.6655 - val_loss: 0.7352 - val_acc: 0.7586
Epoch 7/80
546/546 [==============================] - 279s 511ms/step - loss: 0.9209 - acc: 0.6796 - val_loss: 0.7467 - val_acc: 0.7605


In [22]:
# Save the trained weights in to .h5 format
model.save("./DNST_model_1a.h5")
print("Saved model to disk")


Saved model to disk


In [0]:
from google.colab import files

files.download('./DNST_model_1a.h5')

In [0]:
model.save_weights('DNST_model_1a_wt.h5')

In [0]:
files.download('DNST_model_1a_wt.h5')

In [0]:
from keras.models import load_model
new_model = load_model('DNST_model_1a.h5')

In [0]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import numpy as np

lr_reducer      = ReduceLROnPlateau(monitor='val_acc', factor=np.sqrt(0.1),
                                    cooldown=0, patience=5, min_lr=1e-5)
model_checkpoint= ModelCheckpoint('DNST_model_1a_wt.h5', monitor="val_acc", save_best_only=True,
                                  save_weights_only=True, verbose=1)

callbacks=[lr_reducer, model_checkpoint]


In [51]:
new_model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch= x_train.shape[0] // batch_size, epochs=120,
                    callbacks=callbacks,
                    validation_data = (x_test, y_test),
                    validation_steps = x_test.shape[0] // batch_size, verbose=1)

Epoch 1/120
546/546 [==============================] - 299s 547ms/step - loss: 0.5155 - acc: 0.8221 - val_loss: 0.3819 - val_acc: 0.8916

Epoch 00001: val_acc improved from -inf to 0.89160, saving model to DNST_model_1a_wt.h5
Epoch 2/120
546/546 [==============================] - 291s 533ms/step - loss: 0.5099 - acc: 0.8235 - val_loss: 0.3371 - val_acc: 0.9010

Epoch 00002: val_acc improved from 0.89160 to 0.90100, saving model to DNST_model_1a_wt.h5
Epoch 3/120
546/546 [==============================] - 288s 528ms/step - loss: 0.5037 - acc: 0.8247 - val_loss: 0.3402 - val_acc: 0.9055

Epoch 00003: val_acc improved from 0.90100 to 0.90550, saving model to DNST_model_1a_wt.h5
Epoch 4/120
546/546 [==============================] - 288s 528ms/step - loss: 0.4955 - acc: 0.8285 - val_loss: 0.3428 - val_acc: 0.8993

Epoch 00004: val_acc did not improve from 0.90550
Epoch 5/120
546/546 [==============================] - 288s 528ms/step - loss: 0.4959 - acc: 0.8285 - val_loss: 0.3349 - val_acc

KeyboardInterrupt: ignored

In [52]:
# Save the trained weights in to .h5 format
new_model.save("./DNST_model_1b.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('./DNST_model_1b.h5')
files.download('DNST_model_1a_wt.h5')

In [0]:
new_model.save_weights("DNST_model_1b_wt.h5")

In [0]:
from google.colab import files
final = load_model('./DNST_model_1b.h5')

In [56]:
# Test the model
score = final.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 16s 2ms/step
Test loss: 0.3093938493475318
Test accuracy: 0.909
